In [87]:
import warnings

import numpy as np
import pandas as pd

from jre_utils.datapath import (
    factor_data_paths,
    model_ready_data_paths,
    get_derived_csv_path,
)
from jre_utils.config import asset_types

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [88]:
asset_type = "combined"
dataset_key = "transactions"
years_ahead = 2

metrics = {
    "median": "unit_price_median",
    "gmean": "unit_price_gmean",
}

granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]

metric_key = "gmean"
metric = metrics[metric_key]
metric_pct_chg = metric + "_pct_chg"


In [89]:
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
lfs_revenue_path = factor_data_paths["processed"]["lfs_revenue_breakdown"]["municipality"]

dataset_name = f"sequence_{dataset_key}_{asset_type}_{metric_key}_{years_ahead}"
model_ready_data_path = model_ready_data_paths[dataset_name]

In [90]:
# concal all asset types horizontally here
# Once creating time series dataset,
# create 3 datasets and concat them vertically
# To do that, loop 3 times with different metrics

In [91]:
id_columns = ["area_code", "area", "year"]

factor_log_normalize_columns = [
    "in_migrations",
    "out_migrations",
    "population",
    "taxpayer_count",
    "taxable_income",
    "taxable_income_per_taxpayer",
    "total_tax",
    "new_dwellings",
    "existing_dwellings",

    # Log normalizing seems to be good for everything!
    "total_tax_growth",
    "taxable_income_growth",
    "taxable_income_per_taxpayer_growth",
    "net_migration_ratio",
    "new_dwellings_ratio",
    "taxpayer_count_growth",
]

factor_normalize_columns = [
    "total_tax_growth",
    "taxable_income_growth",
    "taxable_income_per_taxpayer_growth",
    "net_migration_ratio",
    "new_dwellings_ratio",
    "taxpayer_count_growth",
]

factor_maintain_columns = [
    "migrations_is_available",
    "taxable_income_is_available",
    "dwellings_is_available",
    "total_tax_is_available",
]

factor_columns = (
    [f"{column}_log_normalized_yearly" for column in factor_log_normalize_columns]
    + [f"{column}_normalized_yearly" for column in factor_normalize_columns]
    + factor_maintain_columns
)

final_factor_columns = list(set(factor_normalize_columns + factor_log_normalize_columns + factor_columns))

In [92]:
core_log_normalize_columns = ["count", metric, "yearly_price_growth", metric_pct_chg]
core_normalize_columns = ["count_growth", "yearly_price_growth", metric_pct_chg]
core_maintain_columns = ["metric_pct_chg_is_available"]

core_columns = (
    [f"{column}_log_normalized_yearly" for column in core_log_normalize_columns]
    + [f"{column}_normalized_yearly" for column in core_normalize_columns]
    + core_maintain_columns
)

final_core_columns = list(set(core_normalize_columns + core_log_normalize_columns + core_columns))

combined_final_core_columns = [f"{asset_type}_{column}" for column in final_core_columns for asset_type in asset_types]

In [93]:
combined_final_core_columns

['land_yearly_price_growth_log_normalized_yearly',
 'building_yearly_price_growth_log_normalized_yearly',
 'condo_yearly_price_growth_log_normalized_yearly',
 'land_yearly_price_growth_normalized_yearly',
 'building_yearly_price_growth_normalized_yearly',
 'condo_yearly_price_growth_normalized_yearly',
 'land_yearly_price_growth',
 'building_yearly_price_growth',
 'condo_yearly_price_growth',
 'land_unit_price_gmean_pct_chg_log_normalized_yearly',
 'building_unit_price_gmean_pct_chg_log_normalized_yearly',
 'condo_unit_price_gmean_pct_chg_log_normalized_yearly',
 'land_count_growth',
 'building_count_growth',
 'condo_count_growth',
 'land_unit_price_gmean',
 'building_unit_price_gmean',
 'condo_unit_price_gmean',
 'land_unit_price_gmean_log_normalized_yearly',
 'building_unit_price_gmean_log_normalized_yearly',
 'condo_unit_price_gmean_log_normalized_yearly',
 'land_count_log_normalized_yearly',
 'building_count_log_normalized_yearly',
 'condo_count_log_normalized_yearly',
 'land_unit_

In [94]:
final_columns = id_columns + combined_final_core_columns + final_factor_columns

In [95]:
derived_dfs = {}

for asset_type in asset_types:
    df =  pd.read_csv(get_derived_csv_path(asset_type))
    df = df.sort_values(by=group_by_columns, ascending=True)
    df = df[group_by_columns + [metric, "count"]]

    # prepare main metric
    df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change(periods=years_ahead)

    # prepare additional factors
    df["count_growth"] = df.groupby(granularity_columns)["count"].pct_change()
    df["yearly_price_growth"] = df.groupby(granularity_columns)[metric].pct_change()
    df["metric_pct_chg_is_available"] = df[metric_pct_chg].notnull().astype(int)

    for column in core_log_normalize_columns:
        df[f"{column}_log"] = df[column].apply(lambda x: np.log10(1 + x))
        df[f"{column}_log_normalized_yearly"] = df.groupby("year")[f"{column}_log"].transform(
            lambda x: (x - x.mean()) / x.std()
        )

    for column in core_normalize_columns:
        df[f"{column}_normalized_yearly"] = df.groupby("year")[column].transform(
            lambda x: (x - x.mean()) / x.std()
        )

    df = df[group_by_columns + final_core_columns]
    
    derived_dfs[asset_type] = df.rename(
        columns={column: f"{asset_type}_{column}" for column in final_core_columns}
    )

combined_derived_dfs = (
    derived_dfs["building"].merge(
        derived_dfs["land"],
        on=["year", "area_code", "area"],
        how="outer",
    ).merge(
        derived_dfs["condo"],
        on=["year", "area_code", "area"],
        how="outer",
    )
)

In [96]:
combined_derived_dfs

,area,area_code,year,building_yearly_price_growth_log_normalized_yearly,building_yearly_price_growth_normalized_yearly,building_yearly_price_growth,building_unit_price_gmean_pct_chg_log_normalized_yearly,building_count_growth,building_unit_price_gmean,building_unit_price_gmean_log_normalized_yearly,building_count_log_normalized_yearly,building_unit_price_gmean_pct_chg,building_count_growth_normalized_yearly,building_metric_pct_chg_is_available,building_count,building_unit_price_gmean_pct_chg_normalized_yearly,land_yearly_price_growth_log_normalized_yearly,land_yearly_price_growth_normalized_yearly,land_yearly_price_growth,land_unit_price_gmean_pct_chg_log_normalized_yearly,land_count_growth,land_unit_price_gmean,land_unit_price_gmean_log_normalized_yearly,land_count_log_normalized_yearly,land_unit_price_gmean_pct_chg,land_count_growth_normalized_yearly,land_metric_pct_chg_is_available,land_count,land_unit_price_gmean_pct_chg_normalized_yearly,condo_yearly_price_growth_log_normalized_yearly,condo_yearly_price_growth_normalized_yearly,condo_yearly_price_growth,condo_unit_price_gmean_pct_chg_log_normalized_yearly,condo_count_growth,condo_unit_price_gmean,condo_unit_price_gmean_log_normalized_yearly,condo_count_log_normalized_yearly,condo_unit_price_gmean_pct_chg,condo_count_growth_normalized_yearly,condo_metric_pct_chg_is_available,condo_count,condo_unit_price_gmean_pct_chg_normalized_yearly
0,Aichi-ken Agui-cho,23441,2007,NaN,NaN,NaN,NaN,NaN,67590.233575,0.066006,-1.205808,NaN,NaN,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,54723.779572,0.674598,-0.544187,NaN,NaN,0.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aichi-ken Agui-cho,23441,2008,2.521607,3.158972,1.105567,NaN,1.333333,142315.742351,0.968399,-0.674282,NaN,0.497956,0.0,14.0,NaN,-0.742933,-0.581869,-0.250691,NaN,0.500000,41004.994789,0.511063,-0.290165,NaN,0.118376,0.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aichi-ken Agui-cho,23441,2009,0.348771,0.151198,0.024848,2.699531,1.071429,145852.056278,1.103548,-0.183325,1.157887,1.470449,1.0,29.0,3.510887,1.343899,1.331892,0.473600,0.596489,0.233333,60424.944041,0.964866,-0.023743,0.104181,0.479061,1.0,37.0,0.359541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aichi-ken Agui-cho,23441,2010,0.008386,-0.133281,-0.019251,0.337939,0.413793,143044.196187,1.082660,0.055760,0.005119,0.690053,1.0,41.0,0.093175,0.153491,-0.019356,0.044494,1.475480,0.459459,63113.475459,1.015879,0.231435,0.539166,0.515716,1.0,54.0,1.443527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aichi-ken Agui-cho,23441,2011,0.063796,-0.085690,0.010080,0.071647,-0.024390,144486.087473,1.102463,0.100640,-0.009365,0.004318,1.0,40.0,-0.082755,0.338276,0.159122,0.098042,0.444827,-0.203704,69301.255966,1.110362,0.057424,0.146898,-0.533401,1.0,43.0,0.204269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21230,Yamanashi-ken Oshino-mura,19424,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.912959,1.490448,1.101751,2.088592,-0.181818,23205.212513,0.237574,-1.285960,1.366668,-0.415074,1.0,9.0,1.809799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21231,Yamanashi-ken Oshino-mura,19424,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.585510,-0.585584,-0.214747,1.184518,0.333333,18221.953987,0.046115,-1.091118,0.650406,0.438089,1.0,12.0,0.756029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21232,Yamanashi-ken Oshino-mura,19424,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.152940,0.929838,0.612252,0.557395,-0.250000,29378.375210,0.418723,-1.340108,0.266025,-0.791044,1.0,9.0,0.299287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21233,Yamanashi-ken Oshino-mura,19424,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.679992,-1.211575,-0.497405,-0.516146,-0.111111,14765.418036,-0.144001,-1.345537,-0

In [97]:
df = combined_derived_dfs

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)
lfs_revenue_df= pd.read_csv(lfs_revenue_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
    .merge(lfs_revenue_df, on=group_by_columns, how="left")
)

In [98]:
df["migrations_is_available"] = df["net_migration_ratio"].notnull().astype(int)
df["taxable_income_is_available"] = df["taxable_income"].notnull().astype(int)
df["total_tax_is_available"] = df["total_tax"].notnull().astype(int)
df["dwellings_is_available"] = df["new_dwellings"].notnull().astype(int)

In [99]:
taxable_income_df

,year,area_code,area,taxable_income,taxpayer_count,taxpayer_count_growth,taxpayer_count_log,taxpayer_count_log_normalized_yearly,taxpayer_count_growth_log,taxpayer_count_growth_log_normalized_yearly,taxpayer_count_growth_normalized_yearly,taxable_income_per_taxpayer,taxable_income_growth,taxable_income_per_taxpayer_growth,taxable_income_log,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_log,taxable_income_per_taxpayer_log_normalized_yearly,taxable_income_growth_log,taxable_income_growth_log_normalized_yearly,taxable_income_per_taxpayer_growth_log,taxable_income_per_taxpayer_growth_log_normalized_yearly,taxable_income_growth_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly
0,2021,1100,Hokkaido Sapporo-shi,2.971366e+09,906411.0,0.003444,5.957326,2.853349,0.001493,0.335186,0.336425,3278.166013,0.032400,0.028857,9.472956,2.776101,3.515763,0.702540,0.013848,0.150340,0.012355,-0.008686,0.131738,-0.026130
1,2021,1202,Hokkaido Hakodate-shi,3.194402e+08,107534.0,-0.015292,5.031550,1.505360,-0.006693,-0.681591,-0.705828,2970.597039,0.016612,0.032400,8.504390,1.435003,3.472990,0.037166,0.007155,-0.242067,0.013848,0.087863,-0.260901,0.069790
2,2021,1203,Hokkaido Otaru-shi,1.243333e+08,46190.0,-0.013582,4.664557,0.970996,-0.005939,-0.587980,-0.610690,2691.778827,0.028630,0.042793,8.094587,0.867582,3.430201,-0.628452,0.012259,0.057170,0.018198,0.369178,0.037965,0.351160
3,2021,1204,Hokkaido Asahikawa-shi,4.193510e+08,142560.0,0.000681,5.154001,1.683656,0.000296,0.186437,0.182719,2941.575168,0.038224,0.037518,8.622578,1.598649,3.468728,-0.029138,0.016291,0.293581,0.015996,0.226747,0.276577,0.208349
4,2021,1205,Hokkaido Muroran-shi,1.052086e+08,34835.0,-0.010875,4.542028,0.792585,-0.004749,-0.440141,-0.460104,3020.199455,0.009337,0.020434,8.022051,0.767148,3.480179,0.149004,0.004036,-0.424940,0.008785,-0.239547,-0.441825,-0.254155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64371,1985,47361,Okinawa-ken Kumejima-cho,3.992455e+06,1819.0,NaN,3.260071,-1.095376,NaN,NaN,NaN,2194.862562,NaN,NaN,6.601240,-1.037738,3.341605,-0.076708,NaN,NaN,NaN,NaN,NaN,NaN
64372,1985,47362,Okinawa-ken Yaese-cho,9.023653e+06,4997.0,NaN,3.698796,-0.386018,NaN,NaN,NaN,1805.814088,NaN,NaN,6.955382,-0.499835,3.256913,-1.225738,NaN,NaN,NaN,NaN,NaN,NaN
64373,1985,47375,Okinawa-ken Tarama-son,4.395900e+05,196.0,NaN,2.294466,-2.656629,NaN,NaN,NaN,2242.806122,NaN,NaN,5.643049,-2.493124,3.350985,0.050555,NaN,NaN,NaN,NaN,NaN,NaN
64374,1985,47381,Okinawa-ken Taketomi-cho,1.525627e+06,663.0,NaN,2.822168,-1.803406,NaN,NaN,NaN,2301.096531,NaN,NaN,6.183449,-1.672317,3.362124,0.201669,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
df

,area,area_code,year,building_yearly_price_growth_log_normalized_yearly,building_yearly_price_growth_normalized_yearly,building_yearly_price_growth,building_unit_price_gmean_pct_chg_log_normalized_yearly,building_count_growth,building_unit_price_gmean,building_unit_price_gmean_log_normalized_yearly,building_count_log_normalized_yearly,building_unit_price_gmean_pct_chg,building_count_growth_normalized_yearly,building_metric_pct_chg_is_available,building_count,building_unit_price_gmean_pct_chg_normalized_yearly,land_yearly_price_growth_log_normalized_yearly,land_yearly_price_growth_normalized_yearly,land_yearly_price_growth,land_unit_price_gmean_pct_chg_log_normalized_yearly,land_count_growth,land_unit_price_gmean,land_unit_price_gmean_log_normalized_yearly,land_count_log_normalized_yearly,land_unit_price_gmean_pct_chg,land_count_growth_normalized_yearly,land_metric_pct_chg_is_available,land_count,land_unit_price_gmean_pct_chg_normalized_yearly,condo_yearly_price_growth_log_normalized_yearly,condo_yearly_price_growth_normalized_yearly,condo_yearly_price_growth,condo_unit_price_gmean_pct_chg_log_normalized_yearly,condo_count_growth,condo_unit_price_gmean,condo_unit_price_gmean_log_normalized_yearly,condo_count_log_normalized_yearly,condo_unit_price_gmean_pct_chg,condo_count_growth_normalized_yearly,condo_metric_pct_chg_is_available,condo_count,condo_unit_price_gmean_pct_chg_normalized_yearly,population,population_log,population_log_normalized_yearly,in_migrations,out_migrations,net_migration_ratio,in_migrations_log,in_migrations_log_normalized_yearly,out_migrations_log,out_migrations_log_normalized_yearly,net_migration_ratio_log,net_migration_ratio_log_normalized_yearly,net_migration_ratio_normalized_yearly,taxable_income,taxpayer_count,taxpayer_count_growth,taxpayer_count_log,taxpayer_count_log_normalized_yearly,taxpayer_count_growth_log,taxpayer_count_growth_log_normalized_yearly,taxpayer_count_growth_normalized_yearly,taxable_income_per_taxpayer,taxable_income_growth,taxable_income_per_taxpayer_growth,taxable_income_log,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_log,taxable_income_per_taxpayer_log_normalized_yearly,taxable_income_growth_log,taxable_income_growth_log_normalized_yearly,taxable_income_per_taxpayer_growth_log,taxable_income_per_taxpayer_growth_log_normalized_yearly,taxable_income_growth_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly,new_dwellings,existing_dwellings,new_dwellings_ratio,new_dwellings_log,new_dwellings_log_normalized_yearly,existing_dwellings_log,existing_dwellings_log_normalized_yearly,new_dwellings_ratio_log,new_dwellings_ratio_log_normalized_yearly,new_dwellings_ratio_normalized_yearly,total_tax,total_tax_growth,total_tax_log,total_tax_log_normalized_yearly,total_tax_growth_normalized_yearly,total_tax_growth_log,total_tax_growth_log_normalized_yearly,migrations_is_available,taxable_income_is_available,total_tax_is_available,dwellings_is_available
0,Aichi-ken Agui-cho,23441,2007,NaN,NaN,NaN,NaN,NaN,67590.233575,0.066006,-1.205808,NaN,NaN,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,54723.779572,0.674598,-0.544187,NaN,NaN,0.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24979.282722,4.397597,-0.062208,901.0,743.690086,0.006298,2.955207,-0.064155,2.871976,-0.309758,0.002726,1.249193,1.250895,41184220.0,11648.0,0.021754,4.066289,0.091212,0.009347,1.014555,1.022130,3535.733173,0.030296,0.008360,7.614731,0.211655,3.548602,1.247030,0.012962,0.647252,0.003615,0.238545,0.337012,0.101139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3892521.0,0.100463,6.590231,0.180393,0.862704,0.041576,0.909770,1,1,1,0
1,Aichi-ken Agui-cho,23441,2008,2.521607,3.158972,1.105567,NaN,1.333333,142315.742351,0.968399,-0.674282,NaN,0.497956,0.0,14.0,NaN,-0.742933,-0.581869,-0.250691,NaN,0.500000,41004.994789,0.511063,-0.290165,NaN,0.118376,0.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25136.592636,4.400324,-0.060310,940.0,702.479726,0.009449,2.973590,-0.025579,2.84725

In [101]:
df = df[final_columns]

In [102]:
df.to_csv(model_ready_data_path, index=False)

In [28]:
df.describe()

,area_code,year,land_yearly_price_growth_log_normalized_yearly,building_yearly_price_growth_log_normalized_yearly,condo_yearly_price_growth_log_normalized_yearly,land_yearly_price_growth_normalized_yearly,building_yearly_price_growth_normalized_yearly,condo_yearly_price_growth_normalized_yearly,land_yearly_price_growth,building_yearly_price_growth,condo_yearly_price_growth,land_unit_price_gmean_pct_chg_log_normalized_yearly,building_unit_price_gmean_pct_chg_log_normalized_yearly,condo_unit_price_gmean_pct_chg_log_normalized_yearly,land_count_growth,building_count_growth,condo_count_growth,land_unit_price_gmean,building_unit_price_gmean,condo_unit_price_gmean,land_unit_price_gmean_log_normalized_yearly,building_unit_price_gmean_log_normalized_yearly,condo_unit_price_gmean_log_normalized_yearly,land_count_log_normalized_yearly,building_count_log_normalized_yearly,condo_count_log_normalized_yearly,land_unit_price_gmean_pct_chg,building_unit_price_gmean_pct_chg,condo_unit_price_gmean_pct_chg,land_count_growth_normalized_yearly,building_count_growth_normalized_yearly,condo_count_growth_normalized_yearly,land_metric_pct_chg_is_available,building_metric_pct_chg_is_available,condo_metric_pct_chg_is_available,land_count,building_count,condo_count,land_unit_price_gmean_pct_chg_normalized_yearly,building_unit_price_gmean_pct_chg_normalized_yearly,condo_unit_price_gmean_pct_chg_normalized_yearly,in_migrations,taxable_income_log_normalized_yearly,taxable_income_per_taxpayer_growth,new_dwellings_log_normalized_yearly,taxable_income_growth_normalized_yearly,new_dwellings_ratio,net_migration_ratio,total_tax_is_available,population,taxable_income_growth,existing_dwellings_log_normalized_yearly,out_migrations,net_migration_ratio_normalized_yearly,taxable_income_per_taxpayer_growth_normalized_yearly,new_dwellings_ratio_normalized_yearly,migrations_is_available,total_tax_log_normalized_yearly,taxable_income_per_taxpayer_log_normalized_yearly,total_tax,total_tax_growth_normalized_yearly,taxpayer_count_growth_normalized_yearly,taxpayer_count,taxpayer_count_log_normalized_yearly,taxable_income,population_log_normalized_yearly,total_tax_growth,in_migrations_log_normalized_yearly,existing_dwellings,taxpayer_count_growth,out_migrations_log_normalized_yearly,taxable_income_is_available,dwellings_is_available,taxable_income_per_taxpayer,new_dwellings
count,21235.000000,21235.000000,1.939300e+04,1.734200e+04,4.911000e+03,19393.000000,1.734200e+04,4.911000e+03,19393.000000,17342.000000,4911.000000,1.810000e+04,1.617900e+04,4.576000e+03,19393.000000,17342.000000,4911.000000,2.072100e+04,1.854200e+04,5.262000e+03,2.072100e+04,1.854200e+04,5.262000e+03,2.072100e+04,1.854200e+04,5.262000e+03,18100.000000,16179.000000,4576.000000,1.939300e+04,1.734200e+04,4.911000e+03,20721.000000,18542.000000,5262.000000,20721.000000,18542.000000,5262.000000,1.810000e+04,1.617900e+04,4.576000e+03,21234.000000,19917.000000,19917.000000,12175.000000,19917.000000,12175.000000,21234.000000,21235.000000,2.123400e+04,19917.000000,12175.000000,21234.000000,21234.000000,19917.000000,12175.000000,21235.000000,21164.000000,19917.000000,2.116400e+04,21141.000000,19917.000000,1.991700e+04,19917.000000,1.991700e+04,21234.000000,21141.000000,21234.000000,1.217500e+04,19917.000000,21234.000000,21235.000000,21235.000000,19917.000000,12175.000000
mean,21783.687921,2014.446762,1.465565e-18,1.229171e-18,-5.787357e-18,0.000000,3.277789e-18,4.340518e-18,0.051633,0.048543,0.026340,9.814126e-18,4.830935e-18,-1.242208e-17,0.170641,0.166097,0.106521,4.625797e+04,9.127577e+04,2.492731e+05,-2.743276e-17,3.801415e-16,-2.592630e-16,-2.853007e-16,-1.870051e-16,-3.024735e-16,0.039459,0.036415,0.040473,-2.931131e-18,-4.916684e-18,-2.893678e-18,0.873510,0.872560,0.869631,79.942908,90.882483,152.278791,3.140520e-18,8.783519e-19,-7.763797e-19,3933.123293,0.400100,0.001302,-0.108651,0.034121,0.013678,-0.002976,0.996656,9.718990e+04,0.003273,-0.102270,3960.423098,0.099158,-0.012359,-0.082956,0.999953,0.539913,0.1

In [55]:
area_code = 6428
df[df["area_code"] == area_code][["year"] + [f"{asset_type}_yearly_price_growth" for asset_type in asset_types] + [f"{asset_type}_count" for asset_type in asset_types]]

,year,land_yearly_price_growth,building_yearly_price_growth,condo_yearly_price_growth,land_count,building_count,condo_count
17874,2008,-0.148374,NaN,NaN,13.0,12.0,NaN
17875,2009,-0.263423,0.021085,NaN,12.0,14.0,NaN
17876,2010,0.321346,1.464304,NaN,9.0,5.0,NaN
17877,2011,0.113980,-0.349555,NaN,12.0,10.0,NaN
17878,2012,0.062973,1.106908,NaN,17.0,5.0,NaN
17879,2013,-0.132984,-0.630526,NaN,17.0,13.0,NaN
17880,2014,-0.072015,-0.760203,NaN,13.0,4.0,NaN
17881,2015,0.145216,3.453824,NaN,13.0,10.0,NaN
17882,2016,-0.124589,-0.502936,NaN,14.0,4.0,NaN
17883,2017,-0.074654,-0.883946,NaN,12.0,3.0,NaN
